In [21]:
import cv2
import os
import shutil
import imghdr
import numpy as np
import matplotlib.pyplot as plt

In [22]:
# 方案三：统计阈值法
# 原理：基于亮度分布的均值和标准差动态计算阈值
# sigma控制阈值偏移程度（推荐1.5-3.0）
# 自动适应不同亮度分布的图像
# 通过clip防止阈值超出合理范围
def create_mask(imgpath, sigma=1):
    image = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
    """基于统计分布的动态阈值"""   
    mean = np.mean(image)
    std = np.std(image)
    threshold = mean + sigma * std
    threshold = np.clip(threshold, 210, 255)  # 限制阈值范围
    _, mask = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    return mask

In [23]:
def xiufu(imgpath, maskpath):
    src = cv2.imread(imgpath)
    mask = cv2.imread(maskpath, cv2.IMREAD_GRAYSCALE)
    res = cv2.resize(src, None, fx=0.6, fy=0.6, 
                     interpolation=cv2.INTER_CUBIC)
    mask = cv2.resize(mask, None, fx=0.6, fy=0.6, 
                      interpolation=cv2.INTER_CUBIC)
    dst = cv2.inpaint(res, mask, 10, cv2.INPAINT_TELEA)
    return dst

In [24]:
def process_images(rootpath, masksavepath, savepath):
    imgfiles = [f for f in os.listdir(rootpath) if imghdr.what(os.path.join(rootpath, f)) in ['jpeg', 'png', 'gif', 'bmp']]
    
    for imgfile in imgfiles:
        imgpath = os.path.join(rootpath, imgfile)
#         print(imgfile)

        maskpath = os.path.join(masksavepath, "mask_" + imgfile)
        cv2.imwrite(maskpath, create_mask(imgpath))

        dst = xiufu(imgpath, maskpath)
        newname = imgfile.split(".")[0]
        dstresize = cv2.resize(dst,(384,288))
        cv2.imwrite(os.path.join(savepath, newname + ".jpg"), dstresize)

In [25]:

if __name__ == "__main__":
    rootpath = r"F:\wujieyu\img test\TestDataset\Kvasir\images\img"
    masksavepath=r"F:\wujieyu\img test\TestDataset\Kvasir\img Highlights processing FMM\Highlights processing statistical\highlightmask"
    savepath = r"F:\wujieyu\img test\TestDataset\Kvasir\img Highlights processing FMM\Highlights processing statistical\result"

    for directory in [rootpath, masksavepath, savepath]:
        os.makedirs(directory, exist_ok=True)

    process_images(rootpath, masksavepath, savepath)
    print("处理完成")

处理完成
